#### Summarize Dialogue FLAN T5 model
##### Ref:https://www.coursera.org/learn/generative-ai-with-llms

In [1]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

#### https://huggingface.co/datasets/knkarthick/dialogsum

In [8]:
dataset = load_dataset("knkarthick/dialogsum")

In [30]:
dataset.keys()

dict_keys(['train', 'validation', 'test'])

In [38]:
example_indices = [10,20]
for i, index in enumerate(example_indices):
    print('dialogue:')
    print(dataset['test'][index]['dialogue'])
    print()
    print('summary:')
    print(dataset['test'][index]['summary'])
    print()

dialogue:
#Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday

summary:
#Person1# attends Brian's birthday party. Brian thinks #Person1# looks great and charming.

dialogue:
#Person1#: What's wrong with you? Why are you scratching so much?
#Person2#: I feel itchy! I can't stand it anymore! I think I may be coming down with something. I feel lightheaded and weak.
#Person1#: Let me h

In [26]:
model_name='google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [31]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [35]:
prompt = "Ms. Dawson, I need you to take a dictation for me"
prompt_encoded = tokenizer(sentence, return_tensors='pt')
prompt_encoded

{'input_ids': tensor([[  283,     7,     5, 31676,     6,    27,   174,    25,    12,   240,
             3,     9,     3, 12194,   257,    21,   140,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [36]:
prompt_decoded = tokenizer.decode(
        sentence_encoded["input_ids"][0], 
        skip_special_tokens=True
    )
prompt_decoded

'Ms. Dawson, I need you to take a dictation for me'

#### Summarize Dialogue without Prompt Engineering


In [41]:
for i, index in enumerate(example_indices):
    # Extract dialogue and summary for the current example
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']
    
    # Tokenize the dialogue input
    inputs = tokenizer(dialogue, return_tensors='pt')
    
    # Generate a model output with a maximum of 50 new tokens
    generated_output = model.generate(
        inputs["input_ids"], 
        max_new_tokens=50
    )
    
    # Decode the generated output and skip special tokens
    output = tokenizer.decode(
        generated_output[0], 
        skip_special_tokens=True
    )
    
    # Print the input prompt (dialogue), baseline summary, and model output
    print(f'INPUT PROMPT:\n\n{dialogue}\n')
    print(f'BASELINE HUMAN SUMMARY:\n\n{summary}\n')
    print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n\n{output}\n')

INPUT PROMPT:

#Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday

BASELINE HUMAN SUMMARY:

#Person1# attends Brian's birthday party. Brian thinks #Person1# looks great and charming.

MODEL GENERATION - WITHOUT PROMPT ENGINEERING:

Brian, thank you for coming to our party.

INPUT PROMPT:

#Person1#: What's wrong with you? Why are you scratching so much?
#Person2#: I feel itchy! I 

#### Zero Shot Inference with an Instruction Prompt

In [43]:
for i, index in enumerate(example_indices):
    # Extract dialogue and summary for the current example
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt = f"""
    Summarize the following conversation.
    
    {dialogue}
    
    Summary:
        """    
    
    # Tokenize the prompt input
    inputs = tokenizer(prompt, return_tensors='pt')
    
    # Generate a model output with a maximum of 50 new tokens
    generated_output = model.generate(
        inputs["input_ids"], 
        max_new_tokens=50
    )
    
    # Decode the generated output and skip special tokens
    output = tokenizer.decode(
        generated_output[0], 
        skip_special_tokens=True
    )    
    # Print the input prompt (dialogue), baseline summary, and model output
    print(f'INPUT PROMPT:\n\n{dialogue}\n')
    print(f'BASELINE HUMAN SUMMARY:\n\n{summary}\n')
    print(f'MODEL GENERATION - ZERO SHOT INFERENCE:\n\n{output}\n')

INPUT PROMPT:

#Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday

BASELINE HUMAN SUMMARY:

#Person1# attends Brian's birthday party. Brian thinks #Person1# looks great and charming.

MODEL GENERATION - ZERO SHOT INFERENCE:

#Person1#: Happy birthday, Brian. #Person2#: I'm so happy you're having a good time. #Person1#: Thank you, I'm sure you look great today. #

INPUT PROMPT:

#P

#### One Shot Inference with an Instruction Prompt

In [58]:
def make_prompt(example_indices_full, example_index_to_summarize):
    prompt = ''
    for index in example_indices_full:
        dialogue = dataset['test'][index]['dialogue']
        summary = dataset['test'][index]['summary']
        
        # The stop sequence '{summary}\n\n\n' is important for FLAN-T5. Other models may have their own preferred stop sequence.
        prompt += f"""
Dialogue:

{dialogue}

What was going on?
{summary}


"""
    
    dialogue = dataset['test'][example_index_to_summarize]['dialogue']
    
    prompt += f"""
Dialogue:

{dialogue}

What was going on?
"""
        
    return prompt

In [59]:
example_indices_full = [50]
example_index_to_summarize = 10
one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)
print(one_shot_prompt)


Dialogue:

#Person1#: Yeah. Just pull on this strip. Then peel off the back.
#Person2#: You might make a few enemies this way.
#Person1#: If they don't think this is fun, they're not meant to be our friends.
#Person2#: You mean your friends. I think it's cruel.
#Person1#: Yeah. But it's fun. Look at those two ugly old ladies. . . or are they men?
#Person2#: Hurry! Get a shot!. . . Hand it over!
#Person1#: I knew you'd come around. . .

What was going on?
#Person1# is about to make a prank. #Person2# thinks it's cruel at first but then joins.



Dialogue:

#Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I h

In [60]:
# Retrieve the human-generated summary for the selected example
summary = dataset['test'][example_index_to_summarize]['summary']

# Tokenize the one-shot prompt
inputs = tokenizer(one_shot_prompt, return_tensors='pt')

# Generate model output with a max of 50 tokens
generated_output = model.generate(
    inputs["input_ids"], 
    max_new_tokens=50
)

# Decode the model's output and skip special tokens
output = tokenizer.decode(
    generated_output[0], 
    skip_special_tokens=True
)

# Display the results
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT:\n{output}')


---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# attends Brian's birthday party. Brian thinks #Person1# looks great and charming.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT:
Brian's birthday is coming up.


#### Few Shot Inference with an Instruction Prompt

In [64]:
example_indices_full = [40, 80]
example_index_to_summarize = 10

few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(few_shot_prompt)


Dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.



Dialogue:

#Person1#: May, do you mind helping me prepare for the picnic?
#Person2#: Sure. Have you checked the weather report?
#Person1#: Yes. It says it will be sunny all day. No sign of rain at all. This is your father's favorite sausage. Sandwiches for you and Daniel.
#Person2#: No, thanks Mom. I'd like some toast and chicken wings.
#Person1#: Okay. Please take some fruit salad and crackers for me.
#Person2#: Done. Oh, don't forget to take napkins disposable plates, cups and picnic blanket.
#Person1#: All set. 

In [65]:
# Retrieve the human-generated summary for the selected example
summary = dataset['test'][example_index_to_summarize]['summary']

# Tokenize the one-shot prompt
inputs = tokenizer(few_shot_prompt, return_tensors='pt')

# Generate model output with a max of 50 tokens
generated_output = model.generate(
    inputs["input_ids"], 
    max_new_tokens=50
)

# Decode the model's output and skip special tokens
output = tokenizer.decode(
    generated_output[0], 
    skip_special_tokens=True
)

# Display the results
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# attends Brian's birthday party. Brian thinks #Person1# looks great and charming.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT:
Brian's birthday is coming up. Brian's friends are coming over to celebrate.


##### Conclusion
Prompt engineering does slightly better job than no prompt